In [1]:
import numpy as np
import scipy.optimize as scopt

# Exemple de résolution d'un problème en nombres entiers

avec https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.milp.

$$\max x_1$$
avec
$$-x_0 + x_1 \leq 1$$
$$3x_0 + 2x_1 \leq 12$$
$$2x_0 + 3x_1 \leq 12$$
$$-3 \leq x_1 \leq 10$$
$$x_0, x_1 \in \mathbb Z$$

In [2]:
c = -np.array([0, 1])

In [3]:
A = np.array([[-1, 1], [3, 2], [2, 3]])
b_u = np.array([1, 12, 12])
b_l = np.array([-np.inf,-np.inf,-np.inf])
l = [-np.inf,-3]
u = [np.inf,10]

In [4]:
bounds = scopt.Bounds(l,u)
constraints = scopt.LinearConstraint(A, b_l, b_u)
integrality = np.ones_like(c)

In [5]:
res = scopt.milp(c=c, constraints=constraints, integrality=integrality, bounds=bounds)

In [6]:
#une solution optimale
res.x

array([1., 2.])

In [7]:
#valeur de l'optimum
res.fun

-2.0

## sac à dos

Résoudre le problème du sac à dos selon l'exemple ci-dessus avec

| objet  | 1  | 2  | 3  | 4  |
|--------|----|----|----|----|
| valeur | 9  | 11 | 13 | 15 |
| poids  | 6  | 5  | 9  | 7  |

et W = 20

In [8]:
c = -np.array([9, 11, 13, 15])
A = np.array([[6, 5, 9, 7]])
b_u = np.array([20])
b_l = np.array([-np.inf])
l = [0,0,0,0]
u = [1,1,1,1]

bounds = scopt.Bounds(l,u)
constraints = scopt.LinearConstraint(A, b_l, b_u)
integrality = np.ones_like(c)

In [9]:
res=scopt.milp(c=c, constraints=constraints, integrality=integrality, bounds=bounds)

In [10]:
res.x

array([1., 1., 0., 1.])

In [11]:
res.fun

-35.0

# Relaxation linéaire



Résoudre le même problème sans les contraintes d'intégrité (c'est-à-dire que les variables seront supposées réelles dans [0,1] )

In [12]:
c = -np.array([9, 11, 13, 15])
A = np.array([[6, 5, 9, 7]])
b_u = np.array([20])
b_l = np.array([-np.inf])
l = [0,0,0,0]
u = [1,1,1,1]

bounds = scopt.Bounds(l,u)
constraints = scopt.LinearConstraint(A, b_l, b_u)

In [13]:
res=scopt.milp(c=c, constraints=constraints, bounds=bounds)

In [15]:
res.x

array([1.        , 1.        , 0.22222222, 1.        ])

In [14]:
res.fun

-37.888888888888886

# Chargement des fichiers

In [16]:
# sélectionner tous les fichiers sac à dos
from google.colab import files
uploaded = files.upload()

ModuleNotFoundError: No module named 'google.colab'

In [17]:
def open_sac_à_dos(nomfichier):
    """pour un fichier, renvoie les données du sac à dos"""
    f = open(nomfichier)
    lines = f.readlines()
    val = []
    poids = []
    try:
        l = lines[0].strip().split()
        N, W = int(l[0]), int(l[1])
        for i in range(1,N+1):
            l = lines[i].strip().split()
            val.append(int(l[0]))
            poids.append(int(l[1]))

    except BaseException:
        print("Problème Fichier")
        return
    f.close()
    return N,W,val,poids

In [18]:
#exemple
N,W,val,poids = open_sac_à_dos("data/sad_4.txt")

In [20]:
def resoudre_sad_milp(N,W,val,poids):
    """Résout exactement le sac à dos avec le solveur milp"""
    c = -np.array(val)
    A = np.array([poids])
    b_u = np.array([W])
    b_l = np.array([-np.inf])
    l = [0]*N
    u = [1]*N

    bounds = scopt.Bounds(l,u)
    constraints = scopt.LinearConstraint(A, b_l, b_u)
    integrality = np.ones_like(c)
    res=scopt.milp(c=c, constraints=constraints, integrality=integrality, bounds=bounds)
    return -res.fun

In [19]:
def resoudre_relaxation_sad_linear(N,W,val,poids):
    """idem avec la version linéaire"""
    c = -np.array(val)
    A = np.array([poids])
    b_u = np.array([W])
    b_l = np.array([-np.inf])
    l = [0]*N
    u = [1]*N

    bounds = scopt.Bounds(l,u)
    constraints = scopt.LinearConstraint(A, b_l, b_u)
    res=scopt.milp(c=c, constraints=constraints, bounds=bounds)
    return -res.fun


In [21]:
resoudre_sad_milp(*open_sac_à_dos("data/sad_1000.txt"))

54503.0

In [22]:
resoudre_relaxation_sad_linear(*open_sac_à_dos("data/sad_1000.txt"))

54538.04918032787